# lab-12-6 sequence to sequence with attention (Keras + eager version)

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
* [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)
* [Neural Machine Translation with Attention from Tensorflow](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb)

In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os
import pandas as pd
print(tf.__version__)

1.15.0


In [2]:
import re

In [3]:

#path = '/content/drive/My Drive/Colab Notebooks/moviedata2.json'
df = pd.read_json('movies_a.json')


In [4]:
#df = movie_dataframe
def preprocessed_df(df):
    df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))

    df.movie_summary = df.movie_summary.apply(lambda x: str(x).lower())
    df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. ]", '', x))
    df.movie_summary = df.movie_summary.apply(lambda x: x.replace('\\n',''))
    #df.movie_summary = df.movie_summary.apply(lambda x: '<start> ' + x + ' <end>')
    #df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x).lower())
    ##df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))

    df.movie_tagline = df.movie_tagline.apply(lambda x: str(x).lower())
    df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\n','').replace('\\xa0',''))
    df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))
    df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub(' {2,}', '', x))

    #태그라인 없는 데이터 row 삭제
    index = df[df['movie_tagline']==''].index
    df.drop(index, inplace = True)

    #summary 없는 데이터 row 삭제
    summary_index = df[df['movie_summary']==''].index
    df.drop(summary_index, inplace = True)
    
    df.movie_summary = df.movie_summary.apply(lambda x: x.split('.'))
    df = df.reset_index()
    return df.reset_index()



In [5]:
dataframe = preprocessed_df(df)
dataframe

,level_0,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,0,1,"[Crime, Drama, Thriller]",Joker,www.imdb.com/title/tt7286456/mediaviewer/rm335...,[in gotham city mentallytroubled comedian arth...,"[Gotham City, 1981, Arthur Fleck (Joaquin Phoe...",put on a happy face.
1,1,2,"[Action, Adventure, Thriller]",Rambo: Last Blood,www.imdb.com/title/tt1206885/mediaviewer/rm144...,[rambo must confront his past and unearth his ...,[The film opens in Arizona where the aged John...,they drew first blood. he will draw last.
2,2,3,"[Action, Comedy, Crime, Thriller]",Stuber,www.imdb.com/title/tt7734218/mediaviewer/rm293...,[a detective recruits his uber driver into an ...,[The film starts with Detectives Vic Manning (...,buckle up for aride
3,3,4,"[Action, Crime, Thriller]",John Wick: Chapter 3 - Parabellum,www.imdb.com/title/tt6146586/mediaviewer/rm237...,[john wick is on the run after killing a membe...,[John runs through New York as time runs out o...,on may th every action has consequences
4,4,5,"[Comedy, Crime, Drama, Mystery, Thriller]",Knives Out,www.imdb.com/title/tt8946378/mediaviewer/rm256...,[a detective investigates the death of a patri...,"[\n , \n ]",everyone has a motive. no one has a clue.
5,5,6,"[Action, Drama, Horror, Thriller]",Crawl,www.imdb.com/title/tt8364368/mediaviewer/rm325...,[a young woman while attempting to save her fa...,[The film starts at a swim relay at the Univer...,they were here first
6,6,7,"[Adventure, Family, Fantasy]",Maleficent: Mistress of Evil,www.imdb.com/title/tt4777008/mediaviewer/rm354...,[maleficent and her goddaughter aurora begin t...,"[\n , \n ]",on octobergo beyond the fairy tale.
7,7,8,"[Crime, Drama]",The Godfather: Part II,www.imdb.com/title/tt0071562/mediaviewer/rm415...,[the early life and career of vito corleone in...,[The Godfather Part II presents two parallel s...,all the power on earth cant change destiny.
8,8,9,"[Adventure, Drama, Fantasy]",The Lord of the Rings: The Fellowship of the Ring,www.imdb.com/title/tt0120737/mediaviewer/rm359...,[a meek hobbit from the shire and eight compan...,"[The prologue, spoken by Galadriel, shows the ...",its power corrupts all who desire it. only on...
9,9,10,"[Action, Adventure, Fantasy, Sci-Fi]",Star Wars: Episode I - The Phantom Menace,www.imdb.com/title/tt0120915/mediaviewer/rm223...,[two jedi escape a hostile blockade to find al...,[The opening crawl reveals that the Trade Fede...,one truth one hate.


In [6]:
def new_dataframe(df):
    df1 = []
    for idx,val in enumerate(df.movie_summary):
        for val2 in val:
            df1.append([df['movie_name'][idx], val2, df['movie_tagline'][idx]])
    df1 = pd.DataFrame(df1)
    df1.columns = ['movie_name', 'movie_summary', 'movie_tagline']
    #summary 없는 데이터 row 삭제
    summary_index = df1[df1['movie_summary']==''].index
    df1.drop(summary_index, inplace = True)
    
    return df1.reset_index()

In [7]:
df2 = new_dataframe(dataframe)

In [8]:
df2

,index,movie_name,movie_summary,movie_tagline
0,0,Joker,in gotham city mentallytroubled comedian arthu...,put on a happy face.
1,1,Joker,he then embarks on a downward spiral of revol...,put on a happy face.
2,2,Joker,this path brings him facetoface with his alte...,put on a happy face.
3,3,Joker,joker centers around an origin of the iconic ...,put on a happy face.
4,4,Joker,todd phillips exploration of arthur fleck joa...,put on a happy face.
5,5,Joker,arthur fleck works as a clown and is an aspir...,put on a happy face.
6,6,Joker,he has mental health issues part of which inv...,put on a happy face.
7,7,Joker,times are tough and due to his issues and occ...,put on a happy face.
8,8,Joker,over time these issues bear down on him shapi...,put on a happy face.
9,11,Joker,joker,put on a happy face.


In [13]:
def summary_split(df):
    from nltk.corpus import stopwords 
    from nltk.tokenize import word_tokenize 
    
    #단어 3개 미만 문장은 삭제
    df_split = []
    for idx, val in enumerate(df['movie_summary']):
        if len(val)> 3:
            df_split.append([df['movie_name'][idx], val, df['movie_tagline'][idx]])
    df_split = pd.DataFrame(df_split)
    df_split.columns = ['movie_name', 'movie_summary', 'movie_tagline']
    #문장 쪼개기
    df_split['split'] = ""
    stop_words = set(stopwords.words('english')) 
    for idx, i in enumerate(df_split.movie_summary):
        word_tokens = word_tokenize(i)
        result = []
        for w in word_tokens: 
            if w not in stop_words: 
                result.append(w) 
        df_split['split'][idx] = list(result)
        if idx % 1000 == 0:
            print(idx)
    return df_split

In [14]:
dataframe4 = summary_split(df2)
dataframe4

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000


,movie_name,movie_summary,movie_tagline,split
0,Joker,in gotham city mentallytroubled comedian arthu...,put on a happy face.,"[gotham, city, mentallytroubled, comedian, art..."
1,Joker,he then embarks on a downward spiral of revol...,put on a happy face.,"[embarks, downward, spiral, revolution, bloody..."
2,Joker,this path brings him facetoface with his alte...,put on a happy face.,"[path, brings, facetoface, alterego, joker]"
3,Joker,joker centers around an origin of the iconic ...,put on a happy face.,"[joker, centers, around, origin, iconic, arch,..."
4,Joker,todd phillips exploration of arthur fleck joa...,put on a happy face.,"[todd, phillips, exploration, arthur, fleck, j..."
5,Joker,arthur fleck works as a clown and is an aspir...,put on a happy face.,"[arthur, fleck, works, clown, aspiring, standu..."
6,Joker,he has mental health issues part of which inv...,put on a happy face.,"[mental, health, issues, part, involves, uncon..."
7,Joker,times are tough and due to his issues and occ...,put on a happy face.,"[times, tough, due, issues, occupation, arthur..."
8,Joker,over time these issues bear down on him shapi...,put on a happy face.,"[time, issues, bear, shaping, actions, making,..."
9,Joker,joker,put on a happy face.,[joker]


In [15]:
import json

dataframe4.to_json('moviedata_aftersplit.json')

In [16]:
dataframe4

,movie_name,movie_summary,movie_tagline,split
0,Joker,in gotham city mentallytroubled comedian arthu...,put on a happy face.,"[gotham, city, mentallytroubled, comedian, art..."
1,Joker,he then embarks on a downward spiral of revol...,put on a happy face.,"[embarks, downward, spiral, revolution, bloody..."
2,Joker,this path brings him facetoface with his alte...,put on a happy face.,"[path, brings, facetoface, alterego, joker]"
3,Joker,joker centers around an origin of the iconic ...,put on a happy face.,"[joker, centers, around, origin, iconic, arch,..."
4,Joker,todd phillips exploration of arthur fleck joa...,put on a happy face.,"[todd, phillips, exploration, arthur, fleck, j..."
5,Joker,arthur fleck works as a clown and is an aspir...,put on a happy face.,"[arthur, fleck, works, clown, aspiring, standu..."
6,Joker,he has mental health issues part of which inv...,put on a happy face.,"[mental, health, issues, part, involves, uncon..."
7,Joker,times are tough and due to his issues and occ...,put on a happy face.,"[times, tough, due, issues, occupation, arthur..."
8,Joker,over time these issues bear down on him shapi...,put on a happy face.,"[time, issues, bear, shaping, actions, making,..."
9,Joker,joker,put on a happy face.,[joker]


In [17]:

tl = list(dataframe4["movie_tagline"])
tl = ["".join([ j for j in i if j != "\n"]).strip() for i in tl]

dataframe4.movie_tagline = tl

sources = [idx for idx in dataframe4['split']]
targets = [i.split() for i in dataframe4['movie_tagline']]


In [ ]:

# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab

source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}


In [ ]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab


target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}


In [ ]:
import json

json_ = json.dumps(source2idx)
f = open('source2idx.json', 'w')
f.write(json_)
f.close

json_ = json.dumps(idx2source)
f = open('idx2source.json', 'w')
f.write(json_)
f.close

json_ = json.dumps(target2idx)
f = open('target2idx.json', 'w')
f.write(json_)
f.close

json_ = json.dumps(idx2target)
f = open('idx2target.json', 'w')
f.write(json_)
f.close

In [18]:
import json
source2idx = pd.read_json('source2idx.json', typ = 'series')
idx2source = pd.read_json('idx2source.json', typ = 'series')
target2idx = pd.read_json('target2idx.json', typ = 'series')
idx2target = pd.read_json('idx2target.json', typ = 'series')

In [19]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        #각 summary에 있는 단어들을 인덱스로 변환.
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        s_len = list(map(lambda sentence : len(sentence), s_input))
        #길이가 같지 않은 데이터들을 일정한 길이로 맞춰 준다.
        #padding ='post' -> 각 시퀀스 뒤쪽으로 패딩한다
        #maxlen -> 최대 길이를 max_len으로 설정하고 초과한 경우 각 시퀀스의 뒷쪽에서 자른다.
        #summary별 단어 수가 다를것. 이를 맞춰주는 역할이 pad_sequences. 단어 수가 max_len보다 적거나 클경우
        #뒷쪽부터 차이만큼 제거
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    
    elif mode == 'target':
        # preprocessing for target (decoder)
        # 각 태그라인 앞 뒤에 <bos>와 <eos>를 추가
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
         
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

In [20]:
sum_ = 0
for i in dataframe4.movie_summary:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(dataframe4) #summary당 평균 122.84 단어
# preprocessing for source
#단어 수는 120개로 제한
print(average_no)
s_max_len = int(average_no)
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
print('*' * 20, s_input)

21.00398042202846
******************** [[30364 13209 47361 ...     0     0     0]
 [22717 20958 70563 ...     0     0     0]
 [55398  9298 24943 ...     0     0     0]
 ...
 [18071 26985 11882 ...     0     0     0]
 [77118 81818 71650 ...     0     0     0]
 [10523 68934 27168 ...     0     0     0]]


In [21]:
sum_ = 0
for i in dataframe4.movie_tagline:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(dataframe4) #각 tagline별 단어 수는 9.6개
print(average_no)
# preprocessing for target
#단어 수 10개로 제한
t_max_len = int(average_no)
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print('%' * 20, t_input)
print('#' * 20, t_output)

9.66192992123432
%%%%%%%%%%%%%%%%%%%% [[    1 19087 17050 ...     2     0     0]
 [    1 19087 17050 ...     2     0     0]
 [    1 19087 17050 ...     2     0     0]
 ...
 [    1   120  6659 ...   979 20829     2]
 [    1   120  6659 ...   979 20829     2]
 [    1   120  6659 ...   979 20829     2]]
#################### [[19087 17050   120 ...     0     0     0]
 [19087 17050   120 ...     0     0     0]
 [19087 17050   120 ...     0     0     0]
 ...
 [  120  6659   181 ... 20829     2     0]
 [  120  6659   181 ... 20829     2     0]
 [  120  6659   181 ... 20829     2     0]]


# hyper-param

In [22]:
# hyper-parameters
epochs = 100
batch_size = 64
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 256

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
print(data)
data = data.shuffle(buffer_size = buffer_size)
print(data)
data = data.batch(batch_size = batch_size, drop_remainder = True)
print(data)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

<DatasetV1Adapter shapes: ((), (21,), (), (9,), (9,)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>
<DatasetV1Adapter shapes: ((), (21,), (), (9,), (9,)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>
<DatasetV1Adapter shapes: ((64,), (64, 21), (64,), (64, 9), (64, 9)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>


In [23]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [24]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
#         print("state: {}".format(state.shape))
#         print("output: {}".format(state.shape))
              
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [25]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        # * `score = FC(tanh(FC(EO) + FC(H)))`
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
                
        #* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, 1)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        # * `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        # * `embedding output` = The input to the decoder X is passed through an embedding layer.
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        # * `merged vector = concat(embedding output, context vector)`
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [26]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints_attention_after_changed_inputmax'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)

# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)

W1102 03:12:42.242546 28116 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [27]:
EPOCHS = 100
import time
for epoch in range(EPOCHS):
    start = time.time()
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        with tf.GradientTape() as tape:
            
            enc_output, enc_hidden = encoder(s_input, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradient = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradient, variables))
        
    if epoch % 2 == 0:
        #save model every 10 epoch
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
        checkpoint.save(file_prefix = checkpoint_prefix)

Epoch 0 Loss 13420.1201 Batch Loss 5.7605
Time taken for 1 epoch 3882.1318306922913 sec

Epoch 2 Loss 10592.2090 Batch Loss 4.6457
Time taken for 1 epoch 3799.336836338043 sec

Epoch 4 Loss 9030.1562 Batch Loss 4.0358
Time taken for 1 epoch 3723.6380541324615 sec



KeyboardInterrupt: 

In [28]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# result, sentence, attention_plot = evaluate(sentence, encoder, decoder, source2idx, target2idx,
#                                             s_max_len, t_max_len)

In [29]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [30]:

def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
#     attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
#     plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [31]:
#restore checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [32]:
sentence = ' '.join(df.movie_summary[1])


In [33]:
sentence

'in gotham city mentallytroubled comedian arthur fleck is disregarded and mistreated by society  he then embarks on a downward spiral of revolution and bloody crime  this path brings him facetoface with his alterego the joker  joker centers around an origin of the iconic arch nemesis and is an original standalone story not seen before on the big screen  todd phillips exploration of arthur fleck joaquin phoenix a man disregarded by society is not only a gritty character study but also a broader cautionary tale  arthur fleck works as a clown and is an aspiring standup comic  he has mental health issues part of which involves uncontrollable laughter  times are tough and due to his issues and occupation arthur has an even worse time than most  over time these issues bear down on him shaping his actions making him ultimately take on the persona he is more known as   joker  arthur fleck is a wannabe standup comic who suffers from many mental illnesses including one which causes him to laugh 

In [34]:

# sentence = 'tensorflow is a framework for deep learning'

translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

Input: in gotham city mentallytroubled comedian arthur fleck is disregarded and mistreated by society  he then embarks on a downward spiral of revolution and bloody crime  this path brings him facetoface with his alterego the joker  joker centers around an origin of the iconic arch nemesis and is an original standalone story not seen before on the big screen  todd phillips exploration of arthur fleck joaquin phoenix a man disregarded by society is not only a gritty character study but also a broader cautionary tale  arthur fleck works as a clown and is an aspiring standup comic  he has mental health issues part of which involves uncontrollable laughter  times are tough and due to his issues and occupation arthur has an even worse time than most  over time these issues bear down on him shaping his actions making him ultimately take on the persona he is more known as   joker  arthur fleck is a wannabe standup comic who suffers from many mental illnesses including one which causes him to 

In [37]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    idx = 0
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    #print(dec_input)
    xList = []
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        x = tf.nn.top_k(predictions[0], k=4, sorted=True, name=None)
        
        for i in range(4):
            # storing the attention weigths to plot later on
            attention_weights = tf.reshape(attention_weights, (-1, ))
            attention_plot[t] = attention_weights.numpy()

            predicted_id = x[1][i].numpy()

            result += idx2target[predicted_id] + ' '

            if idx2target.get(predicted_id) == '<eos>':
                return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
        xList.append([result])
        result = ''
    return xList

tList=evaluate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
# print(sentence)
from pprint import pprint
pprint(tList)

('<eos> ',
 'in gotham city mentallytroubled comedian arthur fleck is disregarded and '
 'mistreated by society  he then embarks on a downward spiral of revolution '
 'and bloody crime  this path brings him facetoface with his alterego the '
 'joker  joker centers around an origin of the iconic arch nemesis and is an '
 'original standalone story not seen before on the big screen  todd phillips '
 'exploration of arthur fleck joaquin phoenix a man disregarded by society is '
 'not only a gritty character study but also a broader cautionary tale  arthur '
 'fleck works as a clown and is an aspiring standup comic  he has mental '
 'health issues part of which involves uncontrollable laughter  times are '
 'tough and due to his issues and occupation arthur has an even worse time '
 'than most  over time these issues bear down on him shaping his actions '
 'making him ultimately take on the persona he is more known as   joker  '
 'arthur fleck is a wannabe standup comic who suffers from ma

In [ ]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    print(inputs)
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    idx = 0
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    #print(dec_input)
    xList = []
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
        #print("*"*100)
        #print(predictions)
        #print("*"*100)
        #print(tf.argmax(predictions[0]))
        print(predictions)
        x = tf.nn.top_k(predictions[0], k=4, sorted=True, name=None)
        for i in range(4):
            predicted_id = x[1][i].numpy()
            #print(predicted_id)

            result += idx2target[predicted_id] + ' '
            #print("result:", result)
            #print(idx2target[predicted_id])
            #if idx2target.get(predicted_id) == '<eos>':
            #             return result, sentence

          #result=""

        # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)  
        xList.append([result])
        result = ''
        
    return xList

# result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
tList=prediction(sentence1, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
# print(sentence)
from pprint import pprint
pprint(tList)

In [ ]:
df.movie_tagline[8334]

In [ ]:
sentence = df.movie_summary[106].split()[:15]
sentence = ' '.join(sentence)

In [ ]:
sentence

In [ ]:
translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

In [ ]:
sentence = df.movie_summary[107].split()[:30]
sentence = ' '.join(sentence)
sentence

In [ ]:
translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

In [ ]:
df[df['movie_tagline'].str.contains('Past Never Stays in Focus')]

In [ ]:
df.movie_tagline[107]

In [ ]:
df.movie_summary[154]

In [ ]:
sentence = 'will persist in the gulf It’s unlikely to end until the excess capacity in real estate has been cleared'
sentence

In [ ]:
translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)